In [ ]:
import os
import cv2
from albumentations import HorizontalFlip, RandomBrightnessContrast, Compose
from collections import Counter

# 증강할 이미지가 있는 경로
image_folder = "C:/Users/ahsld/Desktop/TOMATO/TOMATO_Train"

# 증강 함수 정의 (회전 없이 좌우 반전과 밝기 조절만 수행)
augment = Compose([
    HorizontalFlip(p=0.5),
    RandomBrightnessContrast(p=0.5)
], bbox_params={'format': 'yolo', 'label_fields': ['class_labels']})

# 클래스 1과 2의 목표 개수 설정
target_count = 10000
class_counts = Counter()

# 현재 클래스 개수 세기
def count_classes_in_folder(folder_path):
    class_counter = Counter()
    for filename in os.listdir(folder_path):
        if filename.endswith('.txt'):
            with open(os.path.join(folder_path, filename), 'r') as file:
                lines = file.readlines()
                for line in lines:
                    class_id = int(float(line.split()[0]))
                    class_counter[class_id] += 1
    return class_counter

# 초기 클래스 개수 확인
class_counts = count_classes_in_folder(image_folder)
print("초기 클래스 개수:", class_counts)

# 증강 수행
for filename in os.listdir(image_folder):
    # "_Gaussian" 또는 "_albu"가 포함된 파일은 건너뛰기
    if "_Gaussian" in filename or "_albu" in filename or not filename.lower().endswith('.png'):
        continue
    
    # 이미지 파일 및 라벨 파일 경로 설정
    file_path = os.path.join(image_folder, filename)
    label_path = os.path.splitext(file_path)[0] + ".txt"

    # 이미지 파일 읽기
    image = cv2.imread(file_path)
    if image is None:
        print(f"이미지를 불러올 수 없습니다: {file_path}")
        continue
    
    # 라벨 파일 확인
    if not os.path.exists(label_path):
        print(f"라벨 파일이 존재하지 않습니다: {label_path}")
        continue

    # 라벨 파일 읽기 (YOLO 형식: class x_center y_center width height)
    with open(label_path, 'r') as f:
        boxes = []
        class_labels = []
        for line in f.readlines():
            parts = line.strip().split()
            class_id = int(float(parts[0]))  # 클래스 ID를 정수로 변환
            bbox = list(map(float, parts[1:]))  # YOLO 형식 (x_center, y_center, width, height)
            
            # 클래스가 1 또는 2인 경우만 증강을 위해 추가
            if class_id in [ 2]:
                boxes.append(bbox)
                class_labels.append(class_id)
    
    # 클래스 1, 2에 해당하는 바운딩 박스가 없으면 건너뜁니다
    if not boxes:
        continue

    # 증강 루프: 목표 개수에 도달할 때까지 증강
    for _ in range(50):  # 한 이미지에서 여러 번 증강 시도
        # 현재 클래스 개수가 목표를 초과하면 종료
        if class_counts[1] >= target_count and class_counts[2] >= target_count:
            break

        # 증강 수행
        try:
            transformed = augment(image=image, bboxes=boxes, class_labels=class_labels)
            augmented_image = transformed['image']
            augmented_bboxes = transformed['bboxes']
            augmented_class_labels = transformed['class_labels']
        except ValueError as e:
            print(f"증강 중 오류가 발생했습니다: {e}. 파일을 건너뜁니다: {file_path}")
            continue

        # 새로운 파일명 생성
        new_filename = os.path.splitext(filename)[0] + f"_albu_{class_counts[1] + class_counts[2]}" + ".png"
        new_file_path = os.path.join(image_folder, new_filename)
        new_label_path = os.path.splitext(new_file_path)[0] + ".txt"

        # 증강된 이미지 저장
        cv2.imwrite(new_file_path, augmented_image)

        # 증강된 라벨 저장 (클래스 ID를 정수로 저장)
        with open(new_label_path, 'w') as f:
            for cls, bbox in zip(augmented_class_labels, augmented_bboxes):
                cls = int(cls)  # 클래스 ID를 정수로 변환하여 저장
                bbox_str = ' '.join(map(str, bbox))
                f.write(f"{cls} {bbox_str}\n")

        # 클래스 개수 업데이트
        for cls in augmented_class_labels:
            class_counts[cls] += 1

        print(f"{new_file_path}에 증강된 이미지 및 라벨 저장 완료")

# 최종 클래스 개수 출력
print("최종 클래스 개수:", class_counts)


c:\Users\ahsld\anaconda3\envs\GGG\lib\site-packages\albumentations\__init__.py:13: UserWarning: A new version of Albumentations is available: 1.4.21 (you have 1.4.18). Upgrade using: pip install -U albumentations. To disable automatic update checks, set the environment variable NO_ALBUMENTATIONS_UPDATE to 1.
  check_for_updates()


초기 클래스 개수: Counter({0: 71127, 1: 1557, 2: 548})
C:/Users/ahsld/Desktop/TOMATO/TOMATO_Train\V001_tom1_00_067_e_12_20220101_27_06100355_38102607_albu_2105.png에 증강된 이미지 및 라벨 저장 완료
C:/Users/ahsld/Desktop/TOMATO/TOMATO_Train\V001_tom1_00_067_e_12_20220101_27_06100355_38102607_albu_2106.png에 증강된 이미지 및 라벨 저장 완료
C:/Users/ahsld/Desktop/TOMATO/TOMATO_Train\V001_tom1_00_067_e_12_20220101_27_06100355_38102607_albu_2107.png에 증강된 이미지 및 라벨 저장 완료
C:/Users/ahsld/Desktop/TOMATO/TOMATO_Train\V001_tom1_00_067_e_12_20220101_27_06100355_38102607_albu_2108.png에 증강된 이미지 및 라벨 저장 완료
C:/Users/ahsld/Desktop/TOMATO/TOMATO_Train\V001_tom1_00_067_e_12_20220101_27_06100355_38102607_albu_2109.png에 증강된 이미지 및 라벨 저장 완료
C:/Users/ahsld/Desktop/TOMATO/TOMATO_Train\V001_tom1_00_067_e_12_20220101_27_06100355_38102607_albu_2110.png에 증강된 이미지 및 라벨 저장 완료
C:/Users/ahsld/Desktop/TOMATO/TOMATO_Train\V001_tom1_00_067_e_12_20220101_27_06100355_38102607_albu_2111.png에 증강된 이미지 및 라벨 저장 완료
C:/Users/ahsld/Desktop/TOMATO/TOMATO_Train\V001_t

KeyboardInterrupt: 

In [2]:
class_counts = count_classes_in_folder(image_folder)
print("클래스 개수:", class_counts)

클래스 개수: Counter({0: 71127, 1: 12780, 2: 1598})
